In [1]:
from bs4 import BeautifulSoup as bs
import requests
from csv import writer

In [2]:
def get_data(url):
    headers = {'user-agent': 'Mozilla/5.0'}
    page = requests.get(url, headers=headers)
    soup = bs(page.content, 'html.parser')
    return soup

In [3]:
def get_next_page(soup):
    container = soup.find("div", class_="pagination spark-pagination-href")
    try: url = "https://www.mediaexpert.pl" + str(container.find_all('a', {'class': 'spark-button is-icon is-simple is-default icon-left'})[1]['href'])
    except Exception as e:
        url = "https://www.mediaexpert.pl" + str(container.find('a', {'class': 'spark-button is-icon is-simple is-default icon-left'})['href'])
    return url

In [4]:
def get_all_pages(url):
    pages = []
    while True:
        pages.append(url)
        soup = get_data(url)
        if not soup.find("i", class_="icon icon-arrow10-right"):
            break
        url = get_next_page(soup)
    return pages

In [5]:
pages = get_all_pages("https://www.mediaexpert.pl/komputery-i-tablety/laptopy-i-ultrabooki/laptopy?limit=50&page=1")[0:20]

In [6]:
def get_all_params(pages):
    params = []
    for page in pages:
        soup = get_data(page)
        boxes = soup.find_all("div", class_ = "column-right")
        for box in boxes:
            screen_params = box.find_all("span", "attribute-values is-regular")[0].text.replace('\n', "").strip("      ").split(",")
            size = screen_params[0]
            resolution = screen_params[1]
            refresh_rate = screen_params[-1]
            processor = str(box.find_all("span", class_ = "attribute-values is-regular")[1].text.replace('\n', "").strip("      "))
            ram = str(box.find_all("span", class_ = "attribute-values is-regular")[2].text.replace('\n', "").strip("      "))
            disc = str(box.find_all("span", class_ = "attribute-values is-regular")[3].text.replace('\n', "").strip("      "))
            graphic_card = str(box.find_all("span", class_ = "attribute-values is-regular")[4].text.replace('\n', "").strip("      "))
            params.extend([size, resolution, refresh_rate, processor, ram , disc, graphic_card])
    return params
    
    

In [7]:
def get_all_prices(pages):
    laptop_prices = []
    i = 0
    for page in pages:
        soup = get_data(page)
        all_prices = soup.find_all("div", class_ = "prices")
        for single_price in all_prices:
            pr = single_price.find_all("span", class_ = "whole")
            for p in pr:
                l = []
                l.append(int(p.text.encode('ascii', 'ignore').decode("utf-8")))
            price = max(l)
            laptop_prices.append(price)
            i = i + 1
            if i%10 == 0:
                break
    return laptop_prices   


In [10]:
def save_to_csv(pages):
    with open('Media Expert laptop prices.csv', 'w', encoding='utf8', newline='') as f:
        thewriter = writer(f)
        header = ['Size', 'Resolution', 'Refresh rate', 'Processor', "RAM", "Disc", "Graphics card", "Price in zl"]
        thewriter.writerow(header)
        for page in pages:
            soup = get_data(page)
            boxes = soup.find_all("div", class_ = "column-right")
            all_prices = soup.find_all("div", class_ = "prices")
            for box, single_price in zip (boxes, all_prices):
                screen_params = box.find_all("span", "attribute-values is-regular")[0].text.replace('\n', "").strip("      ").split(",")
                size = screen_params[0]
                resolution = screen_params[1]
                refresh_rate = screen_params[-1]
                processor = str(box.find_all("span", class_ = "attribute-values is-regular")[1].text.replace('\n', "").strip("      "))
                ram = str(box.find_all("span", class_ = "attribute-values is-regular")[2].text.replace('\n', "").strip("      "))
                disc = str(box.find_all("span", class_ = "attribute-values is-regular")[3].text.replace('\n', "").strip("      "))
                graphic_card = str(box.find_all("span", class_ = "attribute-values is-regular")[4].text.replace('\n', "").strip("      "))      
                pr = single_price.find_all("span", class_ = "whole")
                for p in pr:
                    l = []
                    l.append(int(p.text.encode('ascii', 'ignore').decode("utf-8")))
                price = max(l)
                thewriter.writerow([size, resolution, refresh_rate, processor, ram, disc, graphic_card, price])

    f.close()

In [11]:
save_to_csv(pages)